In [1]:
# Instalasi Library
!pip install pandas nltk requests Sastrawi -q

In [2]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [9]:
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

# 1. Muat Data Mentah
try:
    df = pd.read_csv('df_rosaliarev_raw.csv')
    print("File berhasil dimuat.")
except FileNotFoundError:
    print("Error: File tidak ditemukan. Pastikan file berada di direktori yang sama.")

# Membuat salinan untuk menjaga data asli
df_clean = df.copy()
print(df_clean.head())

File berhasil dimuat.
                                             content  score  thumbsUpCount  \
0  apk ini cukup memuaskan 1. menjauhi calo tiket...      5              0   
1  pihak rosalia indah tidak bertanggung jawab da...      1              0   
2          mantap kalo bisa Adain rute Cilegon lasem      5              0   
3  Aplikasinya bagus. Tapi tolong paging di scree...      2              0   
4  apknya tidak bisa login via nomor telpon,untuk...      3              0   

  reviewCreatedVersion                   at  
0               2.16.0  2025-09-21 12:59:22  
1                  NaN  2025-09-21 03:14:45  
2               2.13.0  2025-09-18 23:33:47  
3               2.16.0  2025-09-15 18:01:52  
4                  NaN  2025-09-14 20:26:29  


In [10]:
# 2. Case Folding (Lowering)
df_clean['cleaned_content'] = df_clean['content'].astype(str).str.lower()
print(df_clean['cleaned_content'].head())

0    apk ini cukup memuaskan 1. menjauhi calo tiket...
1    pihak rosalia indah tidak bertanggung jawab da...
2            mantap kalo bisa adain rute cilegon lasem
3    aplikasinya bagus. tapi tolong paging di scree...
4    apknya tidak bisa login via nomor telpon,untuk...
Name: cleaned_content, dtype: object


In [11]:
# 3. Punctuation & Number Removal
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text) # Hapus semua karakter kecuali huruf dan spasi
    text = re.sub(r'\s+', ' ', text).strip() # Hapus spasi berlebih
    return text

df_clean['cleaned_content'] = df_clean['cleaned_content'].apply(clean_text)

print(df_clean['cleaned_content'].head())

0    apk ini cukup memuaskan menjauhi calo tiket ha...
1    pihak rosalia indah tidak bertanggung jawab da...
2            mantap kalo bisa adain rute cilegon lasem
3    aplikasinya bagus tapi tolong paging di screen...
4    apknya tidak bisa login via nomor telponuntuk ...
Name: cleaned_content, dtype: object


In [13]:
# 4. Tokenization
df_clean['tokens'] = df_clean['cleaned_content'].apply(word_tokenize)

print(df_clean['tokens'].head())

0    [apk, ini, cukup, memuaskan, menjauhi, calo, t...
1    [pihak, rosalia, indah, tidak, bertanggung, ja...
2    [mantap, kalo, bisa, adain, rute, cilegon, lasem]
3    [aplikasinya, bagus, tapi, tolong, paging, di,...
4    [apknya, tidak, bisa, login, via, nomor, telpo...
Name: tokens, dtype: object


In [14]:
# 5. Stopword Removal
stopword_factory = StopWordRemoverFactory()
stopword_remover = stopword_factory.create_stop_word_remover()

def remove_stopwords(tokens):
    return [word for word in stopword_remover.remove(' '.join(tokens)).split()]

df_clean['tokens_no_stopwords'] = df_clean['tokens'].apply(remove_stopwords)

print(df_clean['tokens_no_stopwords'].head())

0    [apk, memuaskan, menjauhi, calo, tiket, harga,...
1    [rosalia, indah, bertanggung, menyalahkan, pel...
2          [mantap, kalo, adain, rute, cilegon, lasem]
3    [aplikasinya, bagus, tolong, paging, screen, h...
4    [apknya, login, via, nomor, telponuntuk, devel...
Name: tokens_no_stopwords, dtype: object


In [15]:
# 6. Stemming
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

def stem_tokens(tokens):
    return [stemmer.stem(word) for word in tokens]

df_clean['tokens_stemmed'] = df_clean['tokens_no_stopwords'].apply(stem_tokens)

print(df_clean['tokens_stemmed'].head())

0    [apk, muas, jauh, calo, tiket, harga, relatif,...
1    [rosalia, indah, tanggung, salah, langgan, mes...
2          [mantap, kalo, adain, rute, cilegon, lasem]
3    [aplikasi, bagus, tolong, paging, screen, hist...
4    [apknya, login, via, nomor, telponuntuk, devel...
Name: tokens_stemmed, dtype: object


In [17]:
# --- FINALISASI ---
# Gabungkan kembali token menjadi kalimat yang sudah bersih
df_clean['final_content'] = df_clean['tokens_stemmed'].apply(lambda x: ' '.join(x))

# Tampilkan perbandingan hasil
print("\n--- Perbandingan Teks Asli dan Teks Bersih (5 Data Pertama) ---")
display(df_clean[['content', 'final_content']].head())

# Checkpoint: Simpan hasil preprocessing ke file CSV baru
FILE_OUTPUT_CLEANED = 'df_rosaliarev_preprocessed.csv'
df_clean.to_csv(FILE_OUTPUT_CLEANED, index=False)
print(f"\nData yang telah diproses disimpan di '{FILE_OUTPUT_CLEANED}'.")


--- Perbandingan Teks Asli dan Teks Bersih (5 Data Pertama) ---


,content,final_content
0,apk ini cukup memuaskan 1. menjauhi calo tiket...,apk muas jauh calo tiket harga relatif murah t...
1,pihak rosalia indah tidak bertanggung jawab da...,rosalia indah tanggung salah langgan mesan via...
2,mantap kalo bisa Adain rute Cilegon lasem,mantap kalo adain rute cilegon lasem
3,Aplikasinya bagus. Tapi tolong paging di scree...,aplikasi bagus tolong paging screen history di...
4,"apknya tidak bisa login via nomor telpon,untuk...",apknya login via nomor telponuntuk developer baik



Data yang telah diproses disimpan di 'df_rosaliarev_preprocessed.csv'.
